# Setup

In [30]:
import nidaqmx 
import atiiaftt as ati
from nidaqmx.stream_readers import AnalogMultiChannelReader
import time as Time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ftsensor
import datetime as dt
import csv
import serial
from time import sleep

sensor = ftsensor.ftsensor(tool_offset=[0,0,52,0,0,0])

ENTER = bytes([0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFC])
EXIT = bytes([0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFD])
ZERO = bytes([0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFE])


# AK70-10 24V Motor Limit
P_MIN = -12.5
P_MAX = 12.5
V_MIN = -50  
V_MAX = 50
T_MIN = -25 
T_MAX = 25
KP_MIN = 0
KP_MAX = 500
KD_MIN = 0
KD_MAX = 5

def unpack(data):
    id = data[0]
    pos = ((data[1]*(P_MAX-P_MIN))/(pow(2,16)-1)) + P_MIN
    vel = ((data[2]*(V_MAX-V_MIN))/(pow(2,12)-1)) + V_MIN
    tor = ((data[3]*(T_MAX-T_MIN))/(pow(2,12)-1)) + T_MIN
    return id, pos, vel, tor

def receive():
    global dT, q1_old, q2_old
    # Read the response from the serial port
    received = ser.readline().decode().split()      

    # Print the received bytes
    # print("Received:", received)

    try:
        test = int(received[0])
        response = [int(x) for x in received]
        [id, p, v, t] = unpack(response)

        if id == 1:
            p = -round(p,3)
            t = -t
            if dT == 0:
                v = 0
            else:
                v = round((p - q1_old)/dT,2)
        
        else:
            p = round(p,3)
            t = t
            if dT == 0:
                v = 0
            else:
                v = round((p-q2_old)/dT,2)

        # print the response from the Arduino
        # print([id, p, v, t])

    except (IndexError, ValueError):
        id = None
        p = 0
        v = 0
        t = 0
    return id, p, v, t

def pack_cmd(p_des, v_des, t_ff, kp, kd):
    bufs = []
    def float_to_uint(val, val_min, val_max, bits):
        val_norm = (val - val_min) / (val_max - val_min)
        val_int = int(round(val_norm * ((1 << bits) - 1)))
        return val_int
    
    p_int = float_to_uint(p_des, P_MIN, P_MAX, 16)
    v_int = float_to_uint(v_des, V_MIN, V_MAX, 12)
    t_int = float_to_uint(t_ff, T_MIN, T_MAX, 12)
    kp_int = float_to_uint(kp, KP_MIN, KP_MAX, 12)
    kd_int = float_to_uint(kd, KD_MIN, KD_MAX, 12)
    
    bufs.append((p_int >> 8) & 0xFF)
    bufs.append(p_int & 0xFF)
    bufs.append((v_int >> 4) & 0xFF)
    bufs.append(((v_int & 0xF) << 4) | ((kp_int >> 8) & 0xF))
    bufs.append(kp_int & 0xFF)
    bufs.append((kd_int >> 4) & 0xFF)
    bufs.append(((kd_int & 0xF) << 4) | ((t_int >> 8) & 0xF))
    bufs.append(t_int & 0xFF)

    return bufs

Construct Serial Connection

In [21]:
# open the serial port
ser = serial.Serial('COM4', 115200, timeout=1)


# Set Zero

In [10]:
print("Begining")


q1_old = 0
pos2_old = 0
dT = 0

#M1
print("Starting Zero Position M1")
ser.write(bytes([0x01])+ZERO)
sleep(3)
[id, p, v, t] = receive()
p = 5
while abs(p) >= 0.001:
    ser.write(bytes([0x01])+ZERO)
    sleep(3)
    [id, p, v, t] = receive()



#M2
print("Starting Zero Position M2")
ser.write(bytes([0x02])+ZERO)
sleep(3)
[id, p, v, t] = receive()
p = 5
while abs(p) >= 0.001:
    ser.write(bytes([0x02])+ZERO)
    sleep(3)
    [id, p, v, t] = receive()

print("Finishing Zero Position")


Begining
Starting Zero Position M1
Starting Zero Position M2
Finishing Zero Position


# Enter Motor and Re-bias Force Sensor

In [31]:
sensor.stop_task()

AttributeError: 'ftsensor' object has no attribute 'task'

In [32]:
##Motor

dT = 0

#M1
print("Starting Enter Motors")
ser.write(bytes([0x01])+ENTER)
[id, p, v, t] = receive()
if id == 1:
    q1 = p
    vel1 = v

elif id == 2:
    q2 = p
    vel2 = v

#M2
ser.write(bytes([0x02])+ENTER)
[id, p, v, t] = receive()

if id == 1:
    q1 = p
    vel1 = v

elif id == 2:
    q2 = p
    vel2 = v

q1_old = q1
q2_old = q2


##Sensor
sensor.start_task()
current_time = Time.time()
duration = 1
values = []
while Time.time()< current_time + duration:
    values.append(np.array(sensor.read_avg_voltage()))

values = np.array(values)
values = [ft.mean() for ft in values.transpose()]
sensor.set_bias(values)
sensor.stop_task()
print(values)

session_parameters = [q1, q2]

Starting Enter Motors
[-0.05162846917882568, 0.4009772479586106, 0.4725998493143258, -0.5946275036144446, -0.39044470581984947, -0.17727290282832758]


Running

In [33]:
def record_session(T1, T2):
    global dT, q1_old, q2_old, q1, q2
    print(f"Starting T1 = {T1} and T2 = {T2}")
    start_time = dt.datetime.today().timestamp()
    t = 0
    l1 = 0.275
    l2 = 0.24

    logs = list()
    motor_logs = list()
    readings = list()

    t_mapping = 1/1.35

    x = l1*np.sin(q1) + l2*np.cos(q2)
    y = l1*np.cos(q1) - l2*np.sin(q2)

    time = 0
    record_duration = 20

    print("Start Controling")
    sensor.start_task()
    while time < record_duration:
        start_time = dt.datetime.today().timestamp()

        log = list()
        log.append(time)

        q1_old = q1
        q2_old = q2

        x_old = x
        y_old = y
        
        log.append(T1)
        log.append(T2)

        command1 = bytes(pack_cmd(0,0,-T1*t_mapping,0,0)) #(pos,vel,torque,kp,kd)
        command2 = bytes(pack_cmd(0,0,T2*t_mapping,0,0)) #(pos,vel,torque,kp,kd)

        #M1
        ser.write(bytes([0x01])+command1)
        [id, p, v, t] = receive()

        if id == 1:
            q1 = p
            vel1 = v
            t1 = t/t_mapping
        
        elif id == 2:
            q2 = p
            vel2 = v
            t2 = t/t_mapping

        #M2
        ser.write(bytes([0x02])+command2)
        [id, p, v, t] = receive()

        if id == 1:
            q1 = p
            vel1 = v
            t1 = t/t_mapping
        
        elif id == 2:
            q2 = p
            vel2 = v
            t2 = t/t_mapping

        log.append(t1)
        log.append(t2)

        log.append(q1)
        log.append(q2)

        # print("q1: ",q1, "\tq2: ", q2)

        x = l1*np.sin(q1) + l2*np.cos(q2)
        y = l1*np.cos(q1) - l2*np.sin(q2)

        log.append(x)
        log.append(y)

        # print("x: ",x, "\ty: ", y)

        if time!= 0:
            x_dot = (x-x_old)/dT
            y_dot = (y-y_old)/dT
        
        else:
            x_dot = 0
            y_dot = 0

        log.append(x_dot)
        log.append(y_dot)

        #Estimate CF
    
        Fx = (t1*l2*np.cos(q2)-t2*l1*np.sin(q1))/(l1*l2*np.cos(q1-q2))
        Fy = (-t1*l2*np.sin(q2)-t2*l1*np.cos(q1))/(l1*l2*np.cos(q1-q2)) 

        log.append(Fx)
        log.append(Fy)

        motor_logs.append(log)

        #Sensor
        reading = sensor.read_ft()
    
        #print(reading)
        
        readings.append(reading)

        log.append(-reading[0])
        log.append(-reading[1])

        logs.append(log)
        
        # print("Progress", f'{round(time/record_duration*100,3)}%')
        dT = dt.datetime.today().timestamp()-start_time
        time+= dT


    print(f"finishing T1 = {T1} and T2 = {T2}")
    return log


In [34]:
Folder = "Y025"
torque_mag = 3
torque_settings = [[0, torque_mag], [torque_mag, 0], [torque_mag, torque_mag],
                   [0, -torque_mag], [-torque_mag, 0], [-torque_mag, -torque_mag]]
logs = []

for session in torque_settings:
    log = record_session(session[0], session[1])
    logs.append(log)
    log_COLUMNS = ["Time", "T1", "T2", "t1", "t2", "q1", "q2", "x", "y", "x_dot", "y_dot", "Fx", "Fy", "Fx_Mea", "Fy_Mea"]

    log_arrays = np.array([np.array(item) for item in log])

    log_arrays_transpose = log_arrays.transpose()
    df_log = pd.DataFrame([])
    for i in range(len(log_COLUMNS)):
        df_log[log_COLUMNS[i]] = log_arrays_transpose[i]

    # print(df_log)
    df_log.to_csv(f"Record_Data/{Folder}/T{session[0]}{session[1]}.csv")


##---Exit Motor---##
#M1
ser.write(bytes([0x01])+EXIT)
receive()

#M2
ser.write(bytes([0x02])+EXIT)
receive()

#Stop Sensor
sensor.stop_task()

Start Controling
x:  0.35555093852611624 	y:  0.24302581207006105
x:  0.36379562539843974 	y:  0.24315384656357752
x:  0.36232008286840844 	y:  0.2438922679030434
x:  0.3608401363966093 	y:  0.24462182274890432
x:  0.36034333150269116 	y:  0.2446230460568385
x:  0.3605780546352139 	y:  0.24354264635285908
x:  0.36057037496416966 	y:  0.2430627078715381
x:  0.3603231721914837 	y:  0.24318318933432087
x:  0.3593331599961578 	y:  0.24366264135462487
x:  0.35883323691575403 	y:  0.2436609181481258
x:  0.35858519683615925 	y:  0.24377966625384767
x:  0.3583370381324986 	y:  0.24389816626015726
x:  0.3568456087103093 	y:  0.24460394959173848
x:  0.35684116896375495 	y:  0.2443639906705654
x:  0.35680949156672925 	y:  0.2429243412959866
x:  0.3558128642819968 	y:  0.24338988661278663
x:  0.35556341711800404 	y:  0.2435056497592274
x:  0.35531385431560764 	y:  0.24362116340066417
x:  0.35531385431560764 	y:  0.24362116340066417
x:  0.3540643144663029 	y:  0.24419498499855025
x:  0.353307099549

# Emergency Exit

In [22]:
ser.write(bytes([0x01])+EXIT)
receive()

#M2
ser.write(bytes([0x02])+EXIT)
receive()


(None, 0, 0, 0)